In [1]:
## remember to run conn_cocolab from the terminal before running cells in this notebook!

import os
import urllib, cStringIO

import pymongo as pm

import matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')

import numpy as np
from __future__ import division
import scipy.stats as stats
import pandas as pd
import json
import re

from PIL import Image
import base64

In [2]:
# directory & file hierarchy
iterationName = 'e1'
exp_path = 'museumdraw'
analysis_dir = os.getcwd()
exp_dir = os.path.abspath(os.path.join(os.getcwd(),'../..','experiments'))
##
sketch_dir = os.path.join(analysis_dir,'sketches')
if not os.path.exists(sketch_dir):
    os.makedirs(sketch_dir)

In [3]:
# set vars 
auth = pd.read_csv('../auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'rxdhawkins.me' ## cocolab ip address

# have to fix this to be able to analyze from local
import pymongo as pm
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['kiddraw']
coll = db['E1c']
collPilot = db['E1']

In [6]:
final_images = coll.find({'dataType':'finalImage'}).sort('time')
imsize = 224
for rec in final_images:
    imgData = rec['imgData'];
    filestr = base64.b64decode(imgData)
    if rec['time']> 1510252452134: 
        if 'age' in rec.keys():
            category_dir = os.path.join(sketch_dir,rec['category'])
            if not os.path.exists(category_dir):
                os.makedirs(category_dir)
            fname = os.path.join(category_dir,'{}_sketch_{}_{}.png'.format(rec['category'], rec['age'],rec['sessionId']))
            with open(fname, "wb") as fh:
                fh.write(imgData.decode('base64'))
                    

{u'_id': ObjectId('5a04f54455a60c1aac24bc7a'),
 u'age': u'10',
 u'category': u'circle',
 u'dataType': u'finalImage',
 u'date': u'2017-11-10T00:39:32.532Z',
 u'imgData': u'iVBORw0KGgoAAAANSUhEUgAAAJYAAACWCAYAAAA8AXHiAAACoUlEQVR4Xu3SMQ0AAAzDsJU/6aHI5wLoEXlnCgQFFny6VODAgiApAFaS1SlYDCQFwEqyOgWLgaQAWElWp2AxkBQAK8nqFCwGkgJgJVmdgsVAUgCsJKtTsBhICoCVZHUKFgNJAbCSrE7BYiApAFaS1SlYDCQFwEqyOgWLgaQAWElWp2AxkBQAK8nqFCwGkgJgJVmdgsVAUgCsJKtTsBhICoCVZHUKFgNJAbCSrE7BYiApAFaS1SlYDCQFwEqyOgWLgaQAWElWp2AxkBQAK8nqFCwGkgJgJVmdgsVAUgCsJKtTsBhICoCVZHUKFgNJAbCSrE7BYiApAFaS1SlYDCQFwEqyOgWLgaQAWElWp2AxkBQAK8nqFCwGkgJgJVmdgsVAUgCsJKtTsBhICoCVZHUKFgNJAbCSrE7BYiApAFaS1SlYDCQFwEqyOgWLgaQAWElWp2AxkBQAK8nqFCwGkgJgJVmdgsVAUgCsJKtTsBhICoCVZHUKFgNJAbCSrE7BYiApAFaS1SlYDCQFwEqyOgWLgaQAWElWp2AxkBQAK8nqFCwGkgJgJVmdgsVAUgCsJKtTsBhICoCVZHUKFgNJAbCSrE7BYiApAFaS1SlYDCQFwEqyOgWLgaQAWElWp2AxkBQAK8nqFCwGkgJgJVmdgsVAUgCsJKtTsBhICoCVZHUKFgNJAbCSrE7BYiApAFaS1SlYDCQFwEqyOgWLgaQAWElWp2AxkBQAK8nqFCwGkgJgJVmdgsVAUgCsJKtTsBhICoCVZHUKFgNJAbCSr

In [374]:
final_images = collPilot.find({'dataType':'finalImage'}).sort('time')
imsize = 224
for rec in final_images:
    imgData = rec['imgData'];
    filestr = base64.b64decode(imgData)
    if rec['trialNum']>0: ## after circle and triangle practice trials
        fname = os.path.join(sketch_dir,'{}_sketch_{}.png'.format(rec['category'], rec['sessionId']))
        with open(fname, "wb") as fh:
            fh.write(imgData.decode('base64'))

In [364]:
rec

{u'_id': ObjectId('5a038cf555a60c1aac24b919'),
 u'category': u'couch',
 u'dataType': u'finalImage',
 u'date': u'2017-11-08T23:02:13.331Z',
 u'imgData': u'...',
 u'sessionId': u'E1_1510182116783',
 u'time': 1510182133331.0,
 u'trialNum': 0}